# Instalasi PyTorch

In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Import Library

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np

# Dataset

In [5]:
df = pd.read_csv("compressed_data.csv")
df = df.dropna(subset=['Cover_Type'])

X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"].values

# Scaling Input & Output

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train - 1, dtype=torch.long)
y_test_tensor = torch.tensor(y_test - 1, dtype=torch.long)

# Training Model

In [9]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train.shape[1]
output_dim = len(set(y))
model = MLPClassifier(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, X_train, y_train, epochs=50):
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

train_model(model, X_train_tensor, y_train_tensor)

Epoch [10/50], Loss: 1.7553
Epoch [20/50], Loss: 1.4289
Epoch [30/50], Loss: 1.1342
Epoch [40/50], Loss: 0.9795
Epoch [50/50], Loss: 0.8748


# Eval

In [13]:
def evaluate_model(model, X_test, y_test):
    with torch.no_grad():
        outputs = model(X_test)
        probs = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(outputs, dim=1).numpy()
        y_test = y_test.numpy()
        acc = accuracy_score(y_test, predictions)
        prec = precision_score(y_test, predictions, average='weighted', zero_division=0)
        rec = recall_score(y_test, predictions, average='weighted')
        f1 = f1_score(y_test, predictions, average='weighted')
        auc = roc_auc_score(y_test, probs.numpy(), multi_class='ovr')
    print(f"Accuracy: {acc:.4f}\nPrecision: {prec:.4f}\nRecall: {rec:.4f}\nF1 Score: {f1:.4f}\nROC AUC: {auc:.4f}")

evaluate_model(model, X_test_tensor, y_test_tensor)

Accuracy: 0.6481
Precision: 0.5926
Recall: 0.6481
F1 Score: 0.6124
ROC AUC: 0.8552


# Instalasi Tensorflow

In [ ]:
!pip install tensorflow

# Import Library

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Training Model

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = keras.utils.to_categorical(y_train - 1, num_classes=len(np.unique(y)))
y_test = keras.utils.to_categorical(y_test - 1, num_classes=len(np.unique(y)))

def build_model(input_dim, output_dim):
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(output_dim, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_dim = X_train.shape[1]
output_dim = y_train.shape[1]
model = build_model(input_dim, output_dim)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.7538 - loss: 0.5900 - val_accuracy: 0.8286 - val_loss: 0.4168
Epoch 2/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.8313 - loss: 0.4027 - val_accuracy: 0.8513 - val_loss: 0.3597
Epoch 3/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.8552 - loss: 0.3504 - val_accuracy: 0.8653 - val_loss: 0.3256
Epoch 4/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.8687 - loss: 0.3188 - val_accuracy: 0.8756 - val_loss: 0.3070
Epoch 5/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.8774 - loss: 0.3015 - val_accuracy: 0.8824 - val_loss: 0.2907
Epoch 6/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 85s 3ms/step - accuracy: 0.8827 - loss: 0.2866 - val_accuracy: 0.8839 - val_loss: 0.2900
Epoch 7/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.8894 - loss: 0.2742 - val_accuracy: 0.8894 - val_loss: 0.2763
Epoch 8/50
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step - accuracy: 

In [20]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovr')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


3632/3632 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
Accuracy: 0.9209
Precision: 0.9217
Recall: 0.9209
F1 Score: 0.9208
ROC AUC: 0.9945
